##Delta Lake Introduction

Databricks&reg; Delta is a transactional storage layer designed specifically to harness the power of Apache Spark and Databricks DBFS. The core abstraction of Databricks Delta is an optimized Spark table that stores your data as Parquet files in DBFS and maintains a transaction log that efficiently tracks changes to the table.

To extract meaningful information out of a Data Lake, we need to resolve problems like:
* Schema enforcement when new tables are introduced 
* Table repairs when any new data is inserted into the data lake
* Frequent refreshes of metadata 
* Bottlenecks of small file sizes for distributed computations
* Difficulty re-sorting data by an index (i.e. userID) if data is spread across many files and partitioned by i.e. eventTime

###The Solution: Databricks Delta

Databricks Delta is a Spark table with built-in reliability and performance optimizations.

You can read and write data stored in Databricks Delta using the same familiar Apache Spark SQL batch and streaming APIs you use to work with Hive tables or DBFS directories. Databricks Delta provides the following functionality:

* <b>ACID transactions</b> - Multiple writers can simultaneously modify a data set and see consistent views.
* <b>DELETES/UPDATES/UPSERTS</b> - Writers can modify a data set without interfering with jobs reading the data set.
* <b>Automatic file management</b> - Data access speeds up by organizing data into large files that can be read efficiently.
* <b>Statistics and data skipping</b> - Reads are 10-100x faster when statistics are tracked about the data in each file, allowing Delta to avoid reading irrelevant information.

-sandbox
#Getting started

Create a table called `customer_delta` we just have to add `DELTA`.

The code looks like:
> `CREATE TABLE <table-name>` <br>
  `USING DELTA` <br>
  `LOCATION <path-do-data> ` <br>

In [5]:
%run ./Reference/Setup

In [6]:
%python
homePath = "dbfs:/Delta lake Demo/Content"
print(homePath)

dbfs:/Delta lake Demo/Content

In [7]:
inputPath = "/mnt/training/online_retail/data-001/data.csv"
genericDataPath = userhome + "/generic/customer-data_01/"
deltaDataPath = userhome + "/delta/customer-data/"
backfillDataPath = userhome + "/delta/backfill-data/"

print (genericDataPath)

dbfs:/user/nagaraj.sengodan@hotmail.com/generic/customer-data_01/

In [8]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from pyspark.sql.functions import col

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

rawDataDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
)

# Previous way of writting files
rawDataDF.write.mode("overwrite").format("parquet").partitionBy("Country").save(genericDataPath)

# write to delta dataset
rawDataDF.write.mode("overwrite").format("delta").partitionBy("Country").save(deltaDataPath)

In [9]:
rawDataDF.take(2)

Out[ 28 ]: [Row(InvoiceNo=536365, StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity=6, InvoiceDate='12/1/10 8:26', UnitPrice=2.55, CustomerID=17850, Country='United Kingdom'),
 Row(InvoiceNo=536365, StockCode='71053', Description='WHITE METAL LANTERN', Quantity=6, InvoiceDate='12/1/10 8:26', UnitPrice=3.39, CustomerID=17850, Country='United Kingdom')]

In [10]:
%python
filePath = homePath + "/delta/customer-data/"
#filePath = "dbfs:/FileStore/tables/"

spark.sql("""
  DROP TABLE IF EXISTS customer_delta
""")
spark.sql("""
  CREATE TABLE customer_delta 
  USING DELTA 
  LOCATION '{}' 
""".format(deltaDataPath))

Out[ 64 ]: DataFrame[]

In [11]:
  %sql
SELECT count(*) FROM customer_data_delta

count(1)
65499


#### Metadata

Since we already have data backing `customer_data_delta` in place, 
the table in the Hive metastore automatically inherits the schema, partitioning, 
and table properties of the existing data. 

Note that we only store table name, path, database info in the Hive metastore,
the actual schema is stored in `_delta_logs`.

In [13]:
%sql
DESCRIBE DETAIL customer_data_delta

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,1b7d5d8e-c87b-47d5-9a19-fcd0c440e21d,nagaraj_sengodan_hotmail_com_db.customer_data_delta,null,dbfs:/user/nagaraj.sengodan@hotmail.com/delta/customer-data,2019-10-16T12:18:23.338+0000,2019-10-16T14:44:26.000+0000,List(Country),39,621086,Map(),1,2


In [14]:
%sql
select * from customer_data_delta limit 10

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541267,22063,CERAMIC BOWL WITH STRAWBERRY DESIGN,6,1/17/11 10:37,2.95,12779,Poland
541267,22624,IVORY KITCHEN SCALES,2,1/17/11 10:37,8.5,12779,Poland
541267,22626,BLACK KITCHEN SCALES,2,1/17/11 10:37,8.5,12779,Poland
541267,22845,VINTAGE CREAM CAT FOOD CONTAINER,2,1/17/11 10:37,6.35,12779,Poland
541267,22178,VICTORIAN GLASS HANGING T-LIGHT,12,1/17/11 10:37,1.25,12779,Poland
541267,37450,CERAMIC CAKE BOWL + HANGING CAKES,6,1/17/11 10:37,2.95,12779,Poland
541267,22510,GINGHAM BABUSHKA DOORSTOP,8,1/17/11 10:37,1.25,12779,Poland
541267,22195,LARGE HEART MEASURING SPOONS,12,1/17/11 10:37,1.65,12779,Poland
541267,22503,CABIN BAG VINTAGE PAISLEY,3,1/17/11 10:37,12.75,12779,Poland
541267,84917,WHITE HAND TOWEL WITH BUTTERFLY,20,1/17/11 10:37,1.25,12779,Poland
